In [ ]:
import pandas as pd
import numpy as np

In [ ]:
movies=pd.read_csv('/content/tmdb_5000_movies.csv')
credits=pd.read_csv('/content/tmdb_5000_credits.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/tmdb_5000_movies.csv'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
movies.head()

In [ ]:
credits.head(1)['cast'].values

In [ ]:
new=movies.merge(credits,on='title')

In [ ]:
new.head()

In [ ]:
#id,gener,keywords,title,overview,cast,crew
new =new[['id','title','overview','keywords','cast','crew']]
new.head()

In [ ]:
#data  preprosessing

In [ ]:
new.isnull().sum()

In [ ]:
new.dropna(inplace=True) #removed null valuues from the data

In [ ]:
new.isnull().sum()

In [ ]:
new.duplicated().sum() #checking fo  duplicate data

In [ ]:
new.iloc[0].keywords

In [ ]:
#the above data is in dictnories,we need to preprocess that into list

In [ ]:
import ast
ast.literal_eval

In [ ]:
def convert(val):
  l=[]
  for i in ast.literal_eval(val):
    l.append(i['name'])
  return l

In [ ]:
#convert keyword,geners to ;ist ysing convert function
new['keywords']=new['keywords'].apply(convert)

In [ ]:
new.head()

In [ ]:
new['cast']=new['cast'].apply(convert)

In [ ]:
new.head()

In [ ]:
#in crew we  need only director name so extract the director name
def ex_dir(dir):
  D=[]
  for i in ast.literal_eval(dir):
    if i['job']=='Director':
      D.append(i['name'])
      break
  return D

In [ ]:
new['crew'].apply(ex_dir)

In [ ]:
new['crew']=new['crew'].apply(ex_dir)

In [ ]:
new.head()

In [ ]:
#convert overview from tring into list
new['overview']=new['overview'].apply(lambda x:x.split())

In [ ]:
new.head()

In [ ]:
#remove the spaces between
#Sam Worthingtonto SamWorthington

In [ ]:
new['cast']=new['cast'].apply(lambda x:[i.replace(" ","") for i in x])
new['crew']=new['crew'].apply(lambda x:[i.replace(" ","") for i in x])
new['keywords']=new['keywords'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
new.head()

In [ ]:
#make cast,crew,keywords,over view int o a single column
new['details']=new['overview']+new['keywords']+new['cast']+new['crew']

In [ ]:
new.head()

In [ ]:
new_mov=new[['id','title','details']]

In [ ]:
new_mov.head()

In [ ]:
#convert details from list in to a paragraph
new_mov['details']=new_mov['details'].apply(lambda x:" ".join(x))

In [ ]:
new_mov.head()

In [ ]:
new_mov.iloc[0].details

In [ ]:
#conver the passage into lowecase
new_mov['details'].apply(lambda x:x.lower())

In [ ]:
new_mov['details']=new_mov['details'].apply(lambda x:x.lower())

In [ ]:
#vectorization
#inorder to recommend movies on movie search,we need check the similarity wors between details of all moves
#inorder to make it simple,insted of making all oword compared,convert the tex into vectors and if distance b/e vectors is small they are similar

In [ ]:
#use bag of word to convert text to vectors

In [ ]:
#using sklearn package

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
#convert to array
vector=cv.fit_transform(new_mov['details']).toarray()

In [ ]:
vector[0]

In [ ]:
cv.get_feature_names_out()

In [ ]:
#apply stemming#
#['eating','ate','eat']->['eat','eat',eat]

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
#stemming eack word
def stemm(wor):
  l=[]
  for i in wor.split():  #convert passage into array
    l.append(ps.stem(i))
  return " ".join(l) #convert array in to pasggage

In [ ]:
new_mov['details']=new_mov['details'].apply(stemm)

In [ ]:
new_mov.head()

In [ ]:
new_mov.iloc[0].details

In [ ]:
#now we need to cal distnance fro a vector to all other vectors
#dist invers to silimarity
#here using euclidean distance is difficut so we use cosine angle between the vectors

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
cosine_similarity(vector)

In [ ]:
cosine_similarity(vector).shape

In [ ]:
similarity=cosine_similarity(vector)

In [ ]:
similarity

In [ ]:
similarity[0]

In [ ]:
#we need to sort the above array and we need to get the top 5 movies,
#but if we soert them the index positions will be changed,so it will not be possible tto get the similarity array using index

In [ ]:
#we use enumerate function
list(enumerate(similarity[0]))

In [ ]:
#sort the enumerated list based on second inde not ist iden


In [ ]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

In [ ]:
def movie_search(name):
  mov_ind=new_mov[new_mov['title']==name].index[0]
  dist=similarity[mov_ind]
  mv_lst=sorted(list(enumerate(dist)),reverse=True,key=lambda x:x[1])[1:6]
  for i in mv_lst:
    print(new_mov.iloc[i[0]].title)

In [ ]:
movie_search('Avatar')

In [ ]:
movie_search('Batman Begins')

In [ ]:
movie_search('Titanic')

In [ ]:
import pickle

In [ ]:
pickle.dump(new_mov,open('movies.pkl','wb'))

NameError: name 'new_mov' is not defined